In [1]:
import torch
import onnx
from onnxsim import simplify
import copy
import numpy as np
import random
from torch.onnx import register_custom_op_symbolic
from torch.onnx.symbolic_helper import parse_args
from models.seg_model_zoo import create_seg_model
import sys 
# sys.path.append(".")
# Path Definitions
# pth_path = '/home/gpu2-user4/access/effvit/checkpoints/effvit/demo_256_256/demo_7411mIoU.pth'
pth_path = '/home/gpu2-user4/access/effvit/checkpoints/effvit/demo_512_512/6225 mIoU_512_512.pth'
onnx_path = '/home/gpu2-user4/access/effvit/onnx/effvit_b0_branch_simp.onnx'
fxp_onnx_path = '/home/gpu2-user4/access/effvit/onnx/fxp_effvit_b0_512_512.onnx'
npz_path = '/home/gpu2-user4/access/effvit/npz_logging'

/home/gpu2-user4/.conda/envs/onnx/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Matplotlib created a temporary config/cache directory at /tmp/matplotlib-cteqaeqt because the default path (/home/gpu2-user4/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
import shutil
import os

def clean_directory(directory):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)

clean_directory(npz_path)


#### Log Scaling Factors

In [3]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    print("CUDA is not available.")
    device = torch.device('cpu')

# 加载模型状态
checkpoint = torch.load(pth_path, map_location=device)
# input = torch.rand(1, 8, 256, 256).to(device)  # 把输入数据移到GPU上
# input = torch.rand(1, 3, 256, 256).to(device)  # 把输入数据移到GPU上
input = torch.rand(1, 3, 512, 512).to(device)  # 把输入数据移到GPU上

# 创建模型并移到GPU上
model = create_seg_model('quan_b0_demo', 'cityscapes').to(device)
try:
    if "state_dict" in checkpoint:
        model.load_state_dict(checkpoint["state_dict"])
    else:
        model.load_state_dict(checkpoint["model_state"])
except RuntimeError as e:
    print(e)
model.eval()

# 进行推理
with torch.no_grad():
    dummy_output = model(input)


/home/gpu2-user4/.conda/envs/onnx/lib/python3.9/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11080). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


CUDA is not available.
idx_concat:  83


#### Start FXP ONNX Conversion

In [4]:
model = onnx.load(onnx_path)
npz_idx = 0
output_map = dict()
input_map = dict()

##### Auxiliary Functions

In [5]:
def has_attribute(attr_list, name):
    has_attr = False
    for a in attr_list:
        if a.name == name:
            has_attr = True
            break
    return has_attr
def get_attribute(attr_list, name):
    for a in attr_list:
        if a.name == name:
            return a
    assert False
def set_output_scale (node, output_scale):
    for i in node.input:
        if i in output_map: # 如果outputmap中有这个input，即和当前node相连的上一个node
            for d in output_map[i]: # 遍历这个input的所有上一个node
                if not has_attribute(d.attribute, "output_scale"): # 如果上一个node没有output_scale
                    attr = onnx.helper.make_attribute("output_bitdepth", 8) # 添加output_bitdepth
                    d.attribute.append(attr)
                    attr = onnx.helper.make_attribute("output_scale", output_scale) # 添加output_scale
                    d.attribute.append(attr) 
                    set_output_scale (d, output_scale) # 向上追溯没有output_scale的node,即输入输出scale相同的node
# Initialization
for node in model.graph.node:
    node.doc_string = ""
    if node.output[0] not in output_map:
        output_map[node.output[0]] = []
    output_map[node.output[0]].append(node)
    for i in node.input:
        if i not in input_map:
            input_map[i] = []
        input_map[i].append(node)
        
for i, node in enumerate(model.graph.node):
    print(f"Node {i}: {node.op_type}")

Node 0: Conv
Node 1: HardSwish
Node 2: Slice
Node 3: Slice
Node 4: Slice
Node 5: Slice
Node 6: Conv
Node 7: HardSwish
Node 8: Conv
Node 9: Add
Node 10: Conv
Node 11: HardSwish
Node 12: Conv
Node 13: Add
Node 14: Conv
Node 15: HardSwish
Node 16: Conv
Node 17: Add
Node 18: Conv
Node 19: HardSwish
Node 20: Conv
Node 21: Add
Node 22: Conv
Node 23: HardSwish
Node 24: Conv
Node 25: HardSwish
Node 26: Conv
Node 27: Conv
Node 28: HardSwish
Node 29: Conv
Node 30: HardSwish
Node 31: Conv
Node 32: Add
Node 33: Conv
Node 34: HardSwish
Node 35: Conv
Node 36: HardSwish
Node 37: Conv
Node 38: Conv
Node 39: HardSwish
Node 40: Conv
Node 41: HardSwish
Node 42: Conv
Node 43: Add
Node 44: Conv
Node 45: HardSwish
Node 46: Conv
Node 47: HardSwish
Node 48: Conv
Node 49: Conv
Node 50: HardSwish
Node 51: Conv
Node 52: HardSwish
Node 53: Conv
Node 54: Add
Node 55: Conv
Node 56: HardSwish
Node 57: Conv
Node 58: HardSwish
Node 59: Conv
Node 60: Conv
Node 61: HardSwish
Node 62: Conv
Node 63: HardSwish
Node 64: Con

In [6]:
for node in model.graph.node:
    if node.op_type == "Conv":
        npz_idx = npz_idx + 1
        val = np.load(npz_path + "/" + str(npz_idx) + "_conv.npz")
        input_scale = val["input_scale"]
        d = node
        if d.input[0] in output_map: # 如果outputmap中有这个input，即和当前node相连的上一个node
            if has_attribute(output_map[d.input[0]][0].attribute, "output_scale"): # 如果上一个node有output_scale
                input_node_out_scale = get_attribute(output_map[d.input[0]][0].attribute, "output_scale").f # 获取上一个node的output_scale
                input_scale[0] = 1/ input_node_out_scale # 计算当前node的input_scale
                assert input_node_out_scale == 1/input_scale[0], str(input_node_out_scale) + "-" + str(1/input_scale[0]) # 检查是否相等
            # 如果上一个node没有output_scale，那么就是输入的scale，那么当前node的input_scale就是1/输入的scale
            set_output_scale (node, 1 / input_scale[0]) # 设置当前node的所有上一个node的output_scale
        # 这里设置上一个node的input_scale 
        if node.input[0] in output_map:
            for d in output_map[node.input[0]]:
                if not has_attribute(d.attribute, "output_scale"):
                    attr = onnx.helper.make_attribute("output_bitdepth", 8)
                    d.attribute.append(attr)
                    attr = onnx.helper.make_attribute("output_scale", 1/input_scale[0])
                    d.attribute.append(attr)  
                
        w = val["w"]
        weight_name = node.input[1]
        ## edit weights
        for idx, t in enumerate(model.graph.initializer):
            if t.name == weight_name:
                value = onnx.helper.make_tensor(t.name, onnx.TensorProto.FLOAT, t.dims, w.flatten())
                model.graph.initializer.remove(t)
                model.graph.initializer.insert(idx, value)
                break
        if "b" in val:
            b = val["b"]
            bias_name = node.input[2]
            value = onnx.helper.make_tensor("bias_" + str(npz_idx), onnx.TensorProto.FLOAT, b.shape, b.flatten())
            model.graph.initializer.append(value)
            node.input[2] = "bias_" + str(npz_idx)
            # ## edit weights
            # for idx, t in enumerate(model.graph.initializer):
            #     if t.name == bias_name:
            #         print (bias_name, t.name, t.dims, b.flatten())
            #         input()
            #         value = onnx.helper.make_tensor(t.name, onnx.TensorProto.FLOAT, t.dims, b.flatten())
            #         model.graph.initializer.remove(t)
            #         model.graph.initializer.insert(idx, value)
            #         break
                
        # del node.attribute[:]
        attr = onnx.helper.make_attribute("input_bitdepth", 8)
        node.attribute.append(attr)
        attr = onnx.helper.make_attribute("input_scale", 1/input_scale[0])
        node.attribute.append(attr)   
        attr = onnx.helper.make_attribute("weight_bitdepth", 8)
        node.attribute.append(attr)
        attr = onnx.helper.make_attribute("weight_ch_scales", (1 / val["weight_scales"]).flatten())
        node.attribute.append(attr)   
        attr = onnx.helper.make_attribute("bias_bitdepth", 16)
        node.attribute.append(attr)
        
    elif node.op_type == "HardSwish":
        npz_idx = npz_idx + 1
        val = np.load(npz_path + "/" + str(npz_idx) + "_pwl.npz")
        
        input_scale = 1/val["input_scale"][0]
        attr = onnx.helper.make_attribute("slopes", val["slopes"])
        node.attribute.append(attr)
        attr = onnx.helper.make_attribute("intercepts", val["intercepts"])
        node.attribute.append(attr)
        attr = onnx.helper.make_attribute("slopes_scale", 1/val["slopes_scale"][0])
        node.attribute.append(attr)
        attr = onnx.helper.make_attribute("intercepts_scale", 1/val["intercepts_scale"][0])
        node.attribute.append(attr)
        attr = onnx.helper.make_attribute("change_pts", val["change_pts"])
        node.attribute.append(attr)
        
        if node.input[0] in output_map:
            for d in output_map[node.input[0]]:
                if not has_attribute(d.attribute, "output_scale"):
                    attr = onnx.helper.make_attribute("output_bitdepth", 8)
                    d.attribute.append(attr)
                    attr = onnx.helper.make_attribute("output_scale", 1/val["input_scale"][0])
                    d.attribute.append(attr)  
        # del node.attribute[:]
        attr = onnx.helper.make_attribute("input_bitdepth", 8)
        node.attribute.append(attr)
        attr = onnx.helper.make_attribute("input_scale", 1/val["input_scale"][0])
        node.attribute.append(attr)  
         
    elif node.op_type == "Add":
        npz_idx = npz_idx + 1
        try:
            val = np.load(npz_path + "/" + str(npz_idx) +  "_add.npz")
        except Exception as e:
            print(f"Error occurred: {e}")  # 打印异常信息
            npz_idx = npz_idx - 1   
            continue
            
        for input1 in node.input:
            if input1 in output_map:
                for d in output_map[input1]:
                    if not has_attribute(d.attribute, "output_scale"):
                        if d.op_type == "Conv":
                            attr = onnx.helper.make_attribute("output_bitdepth", 16)
                        else:
                            attr = onnx.helper.make_attribute("output_bitdepth", 8)
                        d.attribute.append(attr)
                        attr = onnx.helper.make_attribute("output_scale", 1/val["input1_scale"][0])
                        d.attribute.append(attr) 
    elif node.op_type == "Concat":
        npz_idx = npz_idx + 1
        val = np.load(npz_path + "/" + str(npz_idx) +  "_concat.npz")
        for input1 in node.input:
            if input1 in output_map:
                for d in output_map[input1]:
                    if not has_attribute(d.attribute, "output_scale"):
                        attr = onnx.helper.make_attribute("output_bitdepth", 8)
                        d.attribute.append(attr)
                        attr = onnx.helper.make_attribute("output_scale", 1/val["scale_qkv"][0])
                        d.attribute.append(attr) 
        attr = onnx.helper.make_attribute("output_bitdepth", 8)
        node.attribute.append(attr)
        attr = onnx.helper.make_attribute("output_scale", 1/val["scale_qkv"][0])
        node.attribute.append(attr) 
    elif node.op_type == "MatMul":
        # check if input 1 is a constant
        for init in model.graph.initializer:
            if init.name == node.input[1]:
                # get the value as numpy array
                w = onnx.numpy_helper.to_array(init)
                print (w.shape)
                # make a new tensor with the same name, data type, dimensions and values
                
                a = random.randint(0, 255)
                
                value = onnx.helper.make_tensor(init.name + "_" + node.name, onnx.TensorProto.FLOAT, init.dims, a * w.flatten())
                # remove the old initializer and add the modified one
                # model.graph.initializer.remove(init)
                model.graph.initializer.append(value)
                # change the input of the node to point to the new initializer
                node.input[1] = init.name + "_" + node.name
                break
        npz_idx = npz_idx + 1
        print ("npz_logging/" + str(npz_idx) + "_mul.npz")
        val = np.load(npz_path + "/" + str(npz_idx) +  "_mul.npz")
        d = output_map[node.input[0]][0]
        attr = onnx.helper.make_attribute("A_bitdepth", 8)
        node.attribute.append(attr) 
        # attr = copy.deepcopy(get_attribute(d.attribute, "output_scale"))
        # a_scale = attr.f
        # attr.name = "A_scale"
        # node.attribute.append(attr) 
        
        attr = onnx.helper.make_attribute("A_scale", 1/val["input_A_scale"][0])
        node.attribute.append(attr) 
        a_scale=1/val["input_A_scale"][0]
        if node.input[0] in output_map:
            for d in output_map[node.input[0]]:
                if not has_attribute(d.attribute, "output_scale"):
                    attr = onnx.helper.make_attribute("output_bitdepth", 8)
                    d.attribute.append(attr)
                    attr = onnx.helper.make_attribute("output_scale", 1/val["input_A_scale"][0])
                    d.attribute.append(attr) 
        attr = onnx.helper.make_attribute("B_bitdepth", 8)
        node.attribute.append(attr) 
        # d = output_map[node.input[1]][0]
        # attr = copy.deepcopy(get_attribute(d.attribute, "output_scale"))
        # b_scale = attr.f
        # attr.name = "B_scale"
        # node.attribute.append(attr) 
        attr = onnx.helper.make_attribute("B_scale", 1/val["input_B_scale"][0])
        node.attribute.append(attr) 
        b_scale=1/val["input_B_scale"][0]
        if node.input[1] in output_map:
            for d in output_map[node.input[1]]:
                if not has_attribute(d.attribute, "output_scale"):
                    attr = onnx.helper.make_attribute("output_bitdepth", 8)
                    d.attribute.append(attr)
                    attr = onnx.helper.make_attribute("output_scale", 1/val["input_B_scale"][0])
                    d.attribute.append(attr) 
        attr = onnx.helper.make_attribute("output_bitdepth", 8)
        node.attribute.append(attr) 
        attr = onnx.helper.make_attribute("output_scale", 1/val["output_scale"][0])
        assert b_scale * a_scale >= 1/val["output_scale"][0], "scale" + str(b_scale) + " " + str(a_scale) + " " + str(1/val["output_scale"][0])
        
        print ("scale" + str(b_scale) + " " + str(a_scale) + " " + str(1/val["output_scale"][0]))
        
        node.attribute.append(attr) 
    
    elif node.op_type == "Resize":
        npz_idx = npz_idx + 1
        val = np.load(npz_path + "/" + str(npz_idx) + "_resize.npz")
        for input1 in node.input:
            if input1 in output_map:
                for d in output_map[input1]:
                    if not has_attribute(d.attribute, "output_scale"):
                        attr = onnx.helper.make_attribute("output_bitdepth", 8)
                        d.attribute.append(attr)
                        attr = onnx.helper.make_attribute("output_scale", 1/val["input_scale"][0])
                        d.attribute.append(attr) 
        attr = onnx.helper.make_attribute("input_bitdepth", 8)
        node.attribute.append(attr)
        attr = onnx.helper.make_attribute("input_scale", 1/val["input_scale"][0])
        node.attribute.append(attr) 
        attr = onnx.helper.make_attribute("output_bitdepth", 8)
        node.attribute.append(attr)
        attr = onnx.helper.make_attribute("output_scale", 1/val["input_scale"][0])
        node.attribute.append(attr) 
    
    elif node.op_type == "Div":
        npz_idx = npz_idx + 1
    
    elif node.op_type == "ReduceSum":
        None
    else:
        d = output_map[node.input[0]][0]
        if has_attribute(d.attribute, "output_scale"):
            attr = copy.deepcopy(get_attribute(d.attribute, "output_scale"))
            node.attribute.append(attr) 
            attr = onnx.helper.make_attribute("output_bitdepth", 8)
            node.attribute.append(attr)    
        
attr = onnx.helper.make_attribute("output_bitdepth", 8)
node.attribute.append(attr)
attr = onnx.helper.make_attribute("output_scale", 4.0)
node.attribute.append(attr)      

npz_logging/107_mul.npz
scale16.0 8.0 0.0078125
npz_logging/108_mul.npz
scale0.001953125 16.0 0.015625
npz_logging/109_mul.npz
scale0.0078125 16.0 0.00390625
npz_logging/131_mul.npz
scale8.0 8.0 0.0078125
npz_logging/132_mul.npz
scale0.001953125 16.0 0.015625
npz_logging/133_mul.npz
scale0.0078125 16.0 0.00390625
npz_logging/160_mul.npz
scale8.0 4.0 0.015625
npz_logging/161_mul.npz
scale0.0078125 16.0 0.03125
npz_logging/162_mul.npz
scale0.015625 16.0 0.00390625
npz_logging/184_mul.npz
scale8.0 4.0 0.015625
npz_logging/185_mul.npz
scale0.0078125 8.0 0.015625
npz_logging/186_mul.npz
scale0.015625 8.0 0.00390625
Error occurred: [Errno 2] No such file or directory: '/home/gpu2-user4/access/effvit/npz_logging/201_add.npz'
Error occurred: [Errno 2] No such file or directory: '/home/gpu2-user4/access/effvit/npz_logging/201_add.npz'


name: "output_scale"
type: FLOAT
f: 4

##### Save the Final FXP ONNX

In [7]:
onnx.save( model, fxp_onnx_path)